In [2]:

import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
%matplotlib inline
from statsmodels.api import OLS
import statsmodels.api as sm

#preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder 

#sklearn
import sklearn.model_selection as skm
import sklearn.linear_model as skl
from sklearn.model_selection import train_test_split,cross_val_predict
from sklearn.pipeline import Pipeline 
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression 
from sklearn.metrics import accuracy_score, confusion_matrix, jaccard_score, classification_report,\
    log_loss, mean_squared_error
from sklearn.model_selection import train_test_split,  cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV

#tree
from sklearn.tree import DecisionTreeClassifier,plot_tree ,export_text
import sklearn.tree as tree 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from ISLP.bart import BART
#searborn
import seaborn as sns


#Kaggle and loading data
import os
from kaggle.api.kaggle_api_extended import KaggleApi

from ISLP.models import (ModelSpec as MS, summarize , poly)
from functools import partial
from ISLP.models import Stepwise , sklearn_selected , sklearn_selection_path
from l0bnb import fit_path

#SVM
from sklearn.svm import SVC




OSError: [Errno 30] Read-only file system: './germany-used-cars-dataset-2023.zip'

In [ ]:

# Set your Kaggle username and API key
os.environ["KAGGLE_USERNAME"] = "shinle"
os.environ["KAGGLE_KEY"] = "8f3c7c8a0f0afff244ffb147e627b908"

# Specify the dataset you want to download
dataset_name = "wspirat/germany-used-cars-dataset-2023"



# Initialize the Kaggle API
kaggle_api = KaggleApi()
kaggle_api.authenticate()

# Download the dataset
kaggle_api.dataset_download_files(dataset_name, path='.',unzip=True)

# Load the data into a pandas DataFrame
df = pd.read_csv("data.csv")
#df_test= pd.read_csv("test.csv")


ind = df.columns[0]
orignal_df = df
df.head()


In [ ]:
print('Shape before cleaning', df.shape , '\n')

for i in df.columns:
    print('{} column has {} missing values. Data type is {}'.format(i, df[i].isna().sum(), df[i].dtype))
print('')
missing_df = pd.DataFrame(df[df['fuel_consumption_l_100km'].isna()]['fuel_type'].value_counts())
missing_df.columns = ["Missing Count"]
missingCounts = df[df['fuel_consumption_l_100km'].isna()]['fuel_type'].value_counts()
totalCounts = df['fuel_type'].value_counts()
missing_df["Percent Missing"] = [round(missingCounts[i] / totalCounts[i], 2) for i in missing_df.index]

df['fuel_consumption_l_100km'] = df.apply(
        lambda row: float(0) if pd.isnull(row['fuel_consumption_l_100km']) & \
        ((row['fuel_type'] == 'Electric')  or \
         (row['fuel_type'] == 'Hydrogen')  or \
         (row['fuel_type'] == 'Ethanol')) else row['fuel_consumption_l_100km'],
    axis = 1
)

df = df[df['fuel_type'] != 'Unknown']
df = df[df['fuel_type'] != 'Other']
df = df.drop(columns = ind)
df = df.dropna()


missingCounts = df[df['fuel_consumption_l_100km'].isna()]['fuel_type'].value_counts()
totalCounts = df['fuel_type'].value_counts()
missing_df["Percent Missing After"] = [round(missingCounts[i] / totalCounts[i], 2) if i in missingCounts.index else 0 for i in missing_df.index] 
print(missing_df, '\n')


print('Shape after cleaning',df.shape)

In [ ]:
print('Values in the year column before filtering:')
print(df['year'].unique(), '\n')

yearsToFilter = list(df['year'].unique()[:29])
filt = []

for i in range(df.shape[0]):
    filt.append(df['year'].iloc[i] in yearsToFilter)

df = df[filt]
print('Values after filtering:')
print(df['year'].unique())

In [ ]:
#Getting the range from the fuel consumption feature

df['fuel_consumption_l_100km'] = df.apply(
        lambda row: float(row['fuel_consumption_g_km'][:3]) if \
    ('Reichweite' in row['fuel_consumption_g_km']) & (row['fuel_type'] == 'Electric') & ('k' not in row['fuel_consumption_g_km'][:3])\
    else row['fuel_consumption_l_100km'],
    axis = 1
)

filt = [(type(df['fuel_consumption_l_100km'].iloc[i]) != int) and (type(df['fuel_consumption_l_100km'].iloc[i]) != float) for i in range(len(df['fuel_consumption_l_100km']))]

df['fuel_consumption_l_100km'] = df.apply(
        lambda row: float((row['fuel_consumption_l_100km'][:len(row['fuel_consumption_l_100km']) - len(' kWh/100 km')]).replace(',', '.')) \
    if (' kWh/100 km' in str(row['fuel_consumption_l_100km'])) else row['fuel_consumption_l_100km'],
    axis = 1
)

df['fuel_consumption_l_100km'] = df.apply(
        lambda row: 'To Drop' if \
    ((type(row['fuel_consumption_l_100km']) != float) or (row['fuel_consumption_l_100km'] == 0)) & \
    (row['fuel_type'] == 'Electric') else row['fuel_consumption_l_100km'],
    axis = 1
)

df = df[(df['fuel_consumption_l_100km'] != 'To Drop')]
df[df['fuel_type'] == 'Electric'].head()

In [ ]:
df['fuel_consumption_l_100km'] = df.apply(
        lambda row: float((row['fuel_consumption_l_100km'][:len(row['fuel_consumption_l_100km']) - len(' l/100 km')]).replace(',', '.')) \
    if (' l/100 km' in str(row['fuel_consumption_l_100km'])) else row['fuel_consumption_l_100km'],
    axis = 1
)

df['fuel_consumption_l_100km'] = df.apply(
        lambda row: float((row['fuel_consumption_l_100km'][:len(row['fuel_consumption_l_100km']) - len(' kg/100 km')]).replace(',', '.')) \
    if (' kg/100 km' in str(row['fuel_consumption_l_100km'])) else row['fuel_consumption_l_100km'],
    axis = 1
)


df = df[[type(df['fuel_consumption_l_100km'].iloc[i]) == float for i in range(df.shape[0])]]
df.head()

In [ ]:
df['price_in_euro'] = df['price_in_euro'].astype(int)
df['power_kw'] = df['power_kw'].astype(int)
df['fuel_consumption_l_100km'] = df['fuel_consumption_l_100km'].astype(float)

for i in df.columns:
    print('{} column has {} missing values. Data type is {}'.format(i, df[i].isna().sum(), df[i].dtype))
print('')

In [ ]:
newCounts = df['brand'].value_counts()
oriCounts = orignal_df['brand'].value_counts()
percents = pd.Series(newCounts)
for brand in df['brand'].value_counts().index:
    percents[brand] = str(round((newCounts[brand]/oriCounts[brand])*100, 2)) + '%'

counts_df = pd.DataFrame(percents)
counts_df.columns = ['Percent Remaining']
counts_df['Original Count'] = oriCounts.values
counts_df['New Count'] = newCounts.values
counts_df = counts_df.sort_values(by = 'Original Count', ascending = False)
counts_df[:10]